## Day 49 Lecture 1 Assignment

In this assignment, we will apply GMM (Gaussian Mixture Modeling) clustering to a dataset containing player-season statistics for NBA players from the past four years.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.special import entr

This dataset contains player-season statistics for NBA players from the past four years. Each row in this dataset represents a player's per-game averages for a single season. 

This dataset contains the following variables:

- Seas: season ('2019' = 2018-2019 season, '2018' = 2017-2018 season, etc.)
- Player: player name
- Pos: position
- Age: age
- Tm: team
- G: games played
- GS: games started
- MP: minutes played
- FG: field goals
- FGA: field goals attempted
- FG%: field goal percentage
- 3P: 3 pointers
- 3PA: 3 pointers attempted
- 3P%: 3 point percentage
- 2P: 2 pointers
- 2PA: 2 pointers attempted
- 2P%: 2 point percentage
- eFG%: effective field goal percentage
- FT: free throws
- FTA: free throws attempted
- FT%: free throw percentage
- ORB: offensive rebound
- DRB: defensive rebound
- TRB: total rebounds
- AST: assists
- STL: steals
- BLK: blocks
- TOV: turnovers
- PF: personal fouls
- PTS: points

Load the dataset.

In [2]:
# answer goes here
nba = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/nba_player_seasons.csv')
print(nba.shape)
nba.head()

(2141, 30)


,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2019,Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,0.357,1.3,4.1,0.323,0.5,1.0,0.500,0.487,0.4,0.4,0.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,2019,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,0.222,0.2,1.5,0.133,0.2,0.3,0.667,0.278,0.7,1.0,0.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,2019,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,0.345,0.7,2.2,0.338,0.4,1.1,0.361,0.459,0.2,0.3,0.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,0.595,0.0,0.0,0.000,6.0,10.1,0.596,0.595,1.8,3.7,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,0.576,0.0,0.2,0.200,3.4,5.7,0.588,0.579,2.0,2.8,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9


The goal is to cluster these player-seasons to identify potential player "archetypes".  
The pre-processing steps will be identical to what we previously did for K-means.

Begin by removing players whose season did not meet one of the following criteria:
1. Started at least 20 games
2. Averaged at least 10 minutes per game

In [3]:
# answer goes here
#GS = games started; MP = minutes/game
#cleaned data with players meeting criteria
nba_clean = nba.loc[(nba['MP'] >= 10.0) & (nba['GS'] >= 20)].reset_index(drop=True)
nba_clean.shape

(804, 30)

Choose a subset of numeric columns that is interesting to you from an "archetypal" standpoint. 

We will choose the following basic statistics: **points, total rebounds, assists, steals, blocks**, and **turnovers**, but you should feel free to choose other reasonable feature sets if you like. Be careful not to include too many dimensions (curse of dimensionality).

In [4]:
# answer goes here
#pts, trb = rebounds, ast, stl, blk, tov, and adding free throws(FT)
X = nba_clean[['PTS', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'FT']]
X 

,PTS,TRB,AST,STL,BLK,TOV,FT
0,13.9,9.5,1.6,1.5,1.0,1.7,1.8
1,8.9,7.3,2.2,0.9,0.8,1.5,2.0
2,21.3,9.2,2.4,0.5,1.3,1.8,4.3
3,10.9,8.4,1.4,0.5,1.5,1.3,2.5
4,9.4,7.5,1.3,0.8,0.4,0.9,1.9
...,...,...,...,...,...,...,...
799,14.1,2.9,5.8,0.9,0.2,2.3,2.8
800,15.3,2.5,2.5,0.9,0.3,1.6,5.3
801,11.7,6.4,1.4,0.7,1.0,0.8,1.5
802,15.1,9.0,1.8,1.5,0.5,1.9,1.4


Standardize the features in your dataset using scikit-learn's StandardScaler, which will set the mean of each feature to 0 and the variance to 1.

In [5]:
# answer goes here
scale = StandardScaler()
X_scale = pd.DataFrame(scale.fit_transform(X), columns=X.columns)
X_scale

,PTS,TRB,AST,STL,BLK,TOV,FT
0,0.114638,1.612909,-0.617816,1.393220,0.824001,0.023511,-0.281459
1,-0.744985,0.768627,-0.330886,-0.050271,0.420162,-0.218831,-0.155144
2,1.386880,1.497780,-0.235242,-1.012598,1.429758,0.144681,1.297476
3,-0.401136,1.190768,-0.713459,-1.012598,1.833596,-0.461172,0.160643
4,-0.659023,0.845380,-0.761281,-0.290853,-0.387514,-0.945854,-0.218302
...,...,...,...,...,...,...,...
799,0.149023,-0.919937,1.390695,-0.050271,-0.791352,0.750535,0.350115
800,0.355332,-1.073443,-0.187421,-0.050271,-0.589433,-0.097660,1.929050
801,-0.263596,0.423239,-0.713459,-0.531435,0.824001,-1.067025,-0.470931
802,0.320947,1.421027,-0.522172,1.393220,-0.185595,0.265852,-0.534089


Run both K-Means and Gaussian mixtures modeling twice, once with 3 cluster and once with 7 clusters. Print out the resulting means for all 4 scenarios (KM+3, GMM+3, KM+7, GMM+7). When printing the means, transform the scaled versions back into their corresponding unscaled values. 

What "archetypes" do you see? Are the archetypes identified by GMM similar to those identified by K-Means? How do the means of GMM differ from those of K-Means?

In [6]:
print('KM+3')
kmeans3 = KMeans(n_clusters=3)
kmeans3.fit_predict(X_scale)

#inversing scaled features
clusters = kmeans3.cluster_centers_
inverse = pd.DataFrame(scale.inverse_transform(clusters), columns=X.columns)
inverse.style.background_gradient()

KM+3


,PTS,TRB,AST,STL,BLK,TOV,FT
0,14.839860,8.950350,1.996503,0.802098,1.323077,1.774825,2.727273
1,10.300412,4.181481,2.097737,0.803704,0.406996,1.247737,1.431276
2,20.065143,5.410286,5.829143,1.343429,0.508000,2.805714,4.113714


In [7]:
print('GM+3')
gmm3 = GaussianMixture(n_components=3)
gmm3.fit(X_scale)
pred = gmm3.predict(X_scale)
#inversing scaled features
means = gmm3.means_
inverse = pd.DataFrame(scale.inverse_transform(means), columns=X.columns)
inverse.style.background_gradient()

GM+3


,PTS,TRB,AST,STL,BLK,TOV,FT
0,9.830276,4.806520,1.527710,0.789212,0.521838,1.134852,1.372087
1,15.977489,3.777911,4.409724,1.101539,0.303787,2.085537,2.869205
2,16.625693,8.596423,3.639012,0.942920,1.167553,2.276285,3.235278


In [8]:
#Kmeans k=7
print('KM+7')
kmeans7 = KMeans(n_clusters=7)
kmeans7.fit_predict(X_scale)

#inversing scaled transformation
clusters = kmeans7.cluster_centers_
inverse = pd.DataFrame(scale.inverse_transform(clusters), columns=X.columns)
inverse.style.background_gradient()

KM+7


,PTS,TRB,AST,STL,BLK,TOV,FT
0,14.234722,4.493056,6.594444,1.379167,0.425000,2.636111,2.393056
1,17.768519,10.701852,2.200000,0.918519,1.785185,2.144444,3.694444
2,22.260000,5.158750,4.740000,1.236250,0.433750,2.525000,4.807500
3,13.005752,3.958850,2.781416,1.011062,0.326991,1.550885,1.849115
4,7.489947,3.523280,1.357143,0.611640,0.384127,0.880952,0.912169
5,11.769620,7.409494,1.732911,0.687975,0.924051,1.422785,1.924051
6,26.392000,8.540000,7.736000,1.592000,0.868000,4.072000,6.192000


In [9]:
#GMM k=7
print('GM+7')
gmm7 = GaussianMixture(n_components=7, covariance_type='full')
gmm7.fit(X_scale)
gmm7.predict(X_scale)
#inversing
means = gmm7.means_
inverse = pd.DataFrame(scale.inverse_transform(means), columns=X.columns)
inverse.style.background_gradient()

GM+7


,PTS,TRB,AST,STL,BLK,TOV,FT
0,17.716736,4.514359,5.566013,1.397463,0.379585,2.525943,3.531920
1,17.907803,9.741358,2.996646,0.944076,1.450055,2.300681,3.749620
2,12.562102,5.200594,2.040657,1.005109,0.461106,1.428507,1.846161
3,7.240485,3.594083,1.206449,0.637866,0.433349,0.836591,0.811603
4,19.280302,7.136121,5.965805,1.205653,0.640773,2.823694,4.204081
5,13.569126,3.002410,3.661937,0.854886,0.207767,1.690659,1.944704
6,10.584938,7.071378,1.254445,0.574191,1.059277,1.300904,1.744906


*For KM+3 the players with the higher Assists, Steals, Turnovers, Points and Free Throws are grouped together in cluster 1 with the highest of total rebounds and blocks grouped together in Cluster 2. Whereas in GMM+3 the highest cluster centers for AST and STL are in cluster 2 and the rest of the highest centers for the statistics are are grouped in cluster 1 with the second highest centers for AST and STL.*

*In KM+7, the results are somewhat similar to KM+3 where the players with the highest of the 5 (named above) is grouped together in cluster 4 and cluster 6 holds the other 2 statistics highest cluster centers. (Also it seems that cluster 0 in KM+3 and cluster 1 KM+7 have the lowest cluster centers.) In GM+7 all of the highest centers are grouped in cluster 1 except for STL (has 2nd highest). The highest center for STL is in cluster 3.*

Predict the likelihood of each player belonging to one of the 3 clusters using the GMM model. Then, calculate the entropy for each set of predicted probabilities. 

We will use entropy as a measure of how confident we are in the predicted class label. If we had no confidence in our prediction, we would assign 33% probability to each class, while if we were totally confident, we would assign 100% to one class. Entropy would be at a maximum in the "no confidence" scenario and a minimum in the "full confidence" scenario, which makes it a reasonable way to quantify our uncertainty in our prediction. There are certainly other methods as well; feel free to experiment with them if desired.

Which five predicted labels are we least confident about? Which five are we most confident about? Print out the associated details (season, player name, stats, etc.) from those players.

In [10]:
# answer goes here
#GMM k=3
pred = pd.DataFrame(gmm3.predict(X_scale), columns=['pred'])
probs = pd.DataFrame(gmm3.predict_proba(X_scale))
entropy = entr(probs)
total = pd.DataFrame(entropy.sum(axis=1), columns=['Total entropy'])

In [11]:
X = pd.concat([X, pred, probs, total], axis=1)
X

,PTS,TRB,AST,STL,BLK,TOV,FT,pred,0,1,2,Total entropy
0,13.9,9.5,1.6,1.5,1.0,1.7,1.8,0,9.934862e-01,1.724468e-10,0.006514,0.039282
1,8.9,7.3,2.2,0.9,0.8,1.5,2.0,0,9.544124e-01,8.601559e-08,0.045587,0.185313
2,21.3,9.2,2.4,0.5,1.3,1.8,4.3,2,2.840827e-04,2.022047e-16,0.999716,0.002604
3,10.9,8.4,1.4,0.5,1.5,1.3,2.5,0,5.264261e-01,1.133065e-21,0.473574,0.691750
4,9.4,7.5,1.3,0.8,0.4,0.9,1.9,0,9.886841e-01,5.515543e-07,0.011315,0.061970
...,...,...,...,...,...,...,...,...,...,...,...,...
799,14.1,2.9,5.8,0.9,0.2,2.3,2.8,1,1.313541e-15,9.919316e-01,0.008068,0.046924
800,15.3,2.5,2.5,0.9,0.3,1.6,5.3,1,1.610445e-16,9.998359e-01,0.000164,0.001594
801,11.7,6.4,1.4,0.7,1.0,0.8,1.5,0,9.634238e-01,1.963296e-08,0.036576,0.156906
802,15.1,9.0,1.8,1.5,0.5,1.9,1.4,0,9.905530e-01,1.249130e-06,0.009446,0.053457


In [18]:
low5 = X.sort_values(by='Total entropy', ascending=False).head().index

high5 = X.sort_values(by='Total entropy', ascending=True).head().index

In [15]:
low5

,PTS,TRB,AST,STL,BLK,TOV,FT,pred,0,1,2,Total entropy
216,18.9,6.1,2.0,0.6,0.2,1.5,3.4,0,0.649129,0.124153,0.226718,0.875977
468,18.7,6.3,2.8,1.5,0.9,2.5,3.9,1,0.121617,0.664563,0.213821,0.857633
13,11.5,4.6,2.9,0.4,0.5,1.5,1.3,0,0.635769,0.087510,0.276721,0.856643
385,10.2,5.1,3.2,0.8,0.1,1.3,0.5,2,0.048236,0.442266,0.509497,0.850625
604,7.4,2.5,2.6,0.6,0.5,1.9,1.2,1,0.341450,0.625630,0.032919,0.772701


In [21]:
print('The 5 predicted labels (players) least confident on:')
nba_clean.loc[low5]

The 5 predicted labels (players) least confident on:


,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
216,2018,Harrison Barnes,PF,25,DAL,77,77,34.2,7.0,15.7,0.445,1.5,4.3,0.357,5.4,11.4,0.478,0.494,3.4,4.1,0.827,1.0,5.1,6.1,2.0,0.6,0.2,1.5,1.2,18.9
468,2017,Rudy Gay,SF,30,SAC,30,30,33.8,6.7,14.7,0.455,1.4,3.8,0.372,5.3,11.0,0.483,0.502,3.9,4.6,0.855,1.2,5.2,6.3,2.8,1.5,0.9,2.5,2.6,18.7
13,2019,Will Barton,SG,28,DEN,43,38,27.7,4.3,10.7,0.402,1.6,4.6,0.342,2.7,6.1,0.447,0.475,1.3,1.7,0.770,0.7,3.9,4.6,2.9,0.4,0.5,1.5,1.9,11.5
385,2018,Denzel Valentine,SG,24,CHI,77,37,27.2,3.9,9.4,0.417,1.9,4.8,0.386,2.1,4.6,0.450,0.517,0.5,0.7,0.745,0.6,4.5,5.1,3.2,0.8,0.1,1.3,2.4,10.2
604,2017,Isaiah Whitehead,PG,21,BRK,73,26,22.5,2.8,7.0,0.402,0.6,2.0,0.295,2.2,4.9,0.446,0.445,1.2,1.5,0.805,0.4,2.1,2.5,2.6,0.6,0.5,1.9,2.4,7.4


In [16]:
high5

,PTS,TRB,AST,STL,BLK,TOV,FT,pred,0,1,2,Total entropy
60,27.5,13.6,3.7,0.7,1.9,3.5,8.2,2,6.003952e-26,2.176312e-41,1.0,3.486783e-24
454,20.2,7.8,2.1,0.9,2.5,3.8,6.2,2,1.705660e-23,1.067022e-43,1.0,8.942009e-22
6,27.7,12.5,5.9,1.3,1.5,3.7,6.9,2,4.359116e-23,3.779197e-24,1.0,2.448208e-21
405,22.9,8.8,5.4,1.6,1.9,2.9,5.9,2,1.144001e-21,5.836286e-21,1.0,3.270771e-19
252,28.1,11.1,2.3,1.5,2.6,2.2,6.6,2,7.299231e-20,5.851698e-43,1.0,3.216328e-18


In [22]:
print('The 5 predicted labels (players) most confident on:')
nba_clean.loc[high5]

The 5 predicted labels (players) most confident on:


,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
60,2019,Joel Embiid,C,24,PHI,64,64,33.7,9.1,18.7,0.484,1.2,4.1,0.300,7.8,14.6,0.535,0.517,8.2,10.1,0.804,2.5,11.1,13.6,3.7,0.7,1.9,3.5,3.3,27.5
454,2017,Joel Embiid,C,22,PHI,31,31,25.4,6.5,13.8,0.466,1.2,3.2,0.367,5.3,10.7,0.495,0.508,6.2,7.9,0.783,2.0,5.9,7.8,2.1,0.9,2.5,3.8,3.6,20.2
6,2019,Giannis Antetokounmpo,PF,24,MIL,72,72,32.8,10.0,17.3,0.578,0.7,2.8,0.256,9.3,14.5,0.641,0.599,6.9,9.5,0.729,2.2,10.3,12.5,5.9,1.3,1.5,3.7,3.2,27.7
405,2017,Giannis Antetokounmpo,SF,22,MIL,80,80,35.6,8.2,15.7,0.521,0.6,2.3,0.272,7.6,13.5,0.563,0.541,5.9,7.7,0.770,1.8,7.0,8.8,5.4,1.6,1.9,2.9,3.1,22.9
252,2018,Anthony Davis,PF,24,NOP,75,75,36.4,10.4,19.5,0.534,0.7,2.2,0.340,9.7,17.3,0.558,0.552,6.6,8.0,0.828,2.5,8.6,11.1,2.3,1.5,2.6,2.2,2.1,28.1
